In [5]:
import pandas as pd

df_rec=pd.read_excel("data/data_w_functions.xlsx",sheet_name="recipes")
df_quant=pd.read_excel("data/data_w_functions.xlsx",sheet_name="quantity")
df_meas=pd.read_excel("data/data_w_functions.xlsx",sheet_name="measurement",na_values="")
df_ingr=pd.read_excel("data/data_w_functions.xlsx",sheet_name="ingredient")

df_sp_quant=pd.read_excel("data/data_w_functions.xlsx",sheet_name="quantity_sp")
df_sp_meas=pd.read_excel("data/data_w_functions.xlsx",sheet_name="measurement_sp",na_values="")
df_sp_ingr=pd.read_excel("data/data_w_functions.xlsx",sheet_name="ingredient_sp")

df_quant=pd.concat([df_quant,df_sp_quant],axis=1)
df_meas=pd.concat([df_meas,df_sp_meas],axis=1)
df_ingr=pd.concat([df_ingr,df_sp_ingr],axis=1)

In [6]:
def get_recipe(str_recipe):
    return df_rec.at[0,str_recipe]

def get_df_ingr(str_recipe,multiplier=1):
    df_1=df_quant[str_recipe]*multiplier
    df_1=df_1.rename('quantity')
    df_1=pd.to_numeric(df_1)
    df_2=df_meas[str_recipe]
    df_2=df_2.rename('measure')
    df_3=df_ingr[str_recipe]
    df_3=df_3.rename('ingredient_name')

    df_out=pd.concat([df_1,df_2,df_3],axis=1)
    df_out.dropna(axis = 0, how = 'all', inplace = True)
    df_out.drop(df_out[df_out['quantity'] <=0].index, inplace = True)

    return df_out

def get_list(recipe_array,multiplier):

    #multiply individually, then combine
    df_combined=get_df_ingr(recipe_array[0])[0:0]
    for i in range(len(recipe_array)):
        df_combined=pd.concat([df_combined,get_df_ingr(recipe_array[i],multiplier[i])])

    df_combined=df_combined.groupby(['ingredient_name'],as_index=False, sort=False).agg({'quantity': 'sum','measure':'first'})

    #now concatenate columns to get single ingredients list
    df_out = df_combined[['quantity','measure','ingredient_name']]
    df_out.fillna('',inplace=True)
    df_out[df_out['measure']=="None"]=""
    df_out['quantity']=df_out['quantity'].apply(lambda x: f'{x:.0f}' if x%1==0 else str(x))
    df_out['ingredients']=df_out['quantity'] + df_out['measure'] + " " + df_out['ingredient_name']
    df_out=df_out.drop(columns=['quantity','measure','ingredient_name'], axis=1)

    return df_out

def get_spice_pots(recipe_array):
    #multiply individually, then combine
    df_combined=get_df_ingr(recipe_array[0])[0:0]
    df_combined=df_combined.drop(columns=['quantity','measure','ingredient_name'],axis=1)
    for i in range(len(recipe_array)):
        df_i=get_df_ingr(recipe_array[i])
        df_sp_i=df_i.loc[df_i['ingredient_name'].str.contains('spice pot')]
        if len(df_sp_i)>0:
            #check if we have spice pot:
                if df_sp_i['ingredient_name'].iat[0] in df_ingr.columns:
                    ingred_list_i=get_list([df_sp_i['ingredient_name'].iat[0]],[1])
                    ingred_list_i.rename(columns={"ingredients":(str(1) + " " + df_sp_i['ingredient_name'].iat[0]) }, inplace=True)
                    df_combined=pd.concat([df_combined,ingred_list_i],axis=1)
                else:
                    df_combined[(str(1) + " " + df_sp_i['ingredient_name'].iat[0])]=""
                    df_combined[(str(1) + " " + df_sp_i['ingredient_name'].iat[0])].iat[0]="We do not have this, please check original card"

    df_combined.fillna('',inplace=True)
    return df_combined

def get_sainsburys_link(recipe_array,multiplier):
    #multiply individually, then combine
    df_combined=get_df_ingr(recipe_array[0])[0:0]
    for i in range(len(recipe_array)):
        df_combined=pd.concat([df_combined,get_df_ingr(recipe_array[i],multiplier[i])])

    df_combined=df_combined.groupby(['ingredient_name'],as_index=False, sort=False).agg({'quantity': 'sum','measure':'first'})

    #now concatenate columns to get single ingredients list
    df_out = df_combined[['quantity','measure','ingredient_name']]
    df_out.fillna('',inplace=True)
    df_out=df_out.drop(columns=['quantity','measure'], axis=1)
    ls_out=df_out['ingredient_name'].tolist()
    str_out=",".join(ls_out)
    sains_link="https://www.sainsburys.co.uk/gol-ui/SearchResults/"
    return sains_link + str_out

In [11]:
get_recipe('Greek Mushroom Ragu & Olive Oil Mash')

"1. Peel, halve and finely slice the onion. Peel and finely chop the garlic. Slice the mushrooms ( 1 cm ). Halve the olives.\n2. Boil a kettle. Peel and chop the potatoes into 2-3cm chunks. Transfer the potatoes to a medium saucepan. Add 2 good pinches of salt. Cover well with boiled water. Bring to a low boil. Cook for 8-10 mins, until tender, while you continue.\n3. Heat 2 tbsp of oil in a large frying pan. Gently fry the onion for 5 mins, stirring now and then. Increase the heat, add the mushrooms and cook for 4 mins, until starting to soften.\n4. Drain the cooked potatoes, then transfer them back to their saucepan. Set aside for now. Drain and rinse the lentils.\n5. Stir the garlic and Greek spice pot into the mushrooms.\nPour in the wine. Let it bubble and reduce by 1/2. Add the lentils and passata. Fill the empty passata pot with water. Add that. Season. Bring to a low - medium boil. Cook for 5 mi\xadns. Stir often and keep an eye on the liquid - you don't want ' it too sloppy bu

In [12]:
get_df_ingr('Greek Mushroom Ragu & Olive Oil Mash')

,quantity,measure,ingredient_name
0,1.0,NaN,red onion
1,2.0,cloves,garlic
2,200.0,g,portobello mushrooms
3,60.0,g,green olives
4,500.0,g,potatoes
5,1.0,tin,dark green lentils
6,1.0,NaN,greek spice pot
7,100.0,ml,red wine
8,200.0,ml,passata
9,15.0,g,parsley
